<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-100f20af-1b36-5adc-0795-f06b020c050b)


In [3]:
# !git clone https://github.com/MsAsterisk1/ERM_GDRO.git
# %cd /content/ERM_GDRO

In [4]:
# !git pull

In [5]:
# !pip install transformers

In [3]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier

Now lets try getting Bert working

In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [8]:
df = get_CivilComments_df()

In [18]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [16]:
tr,cv,test = get_CivilComments_DataLoaders(datasets=datasets)

This is how we would initialize model

In [13]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, tr, cv, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM')


Epoch 1 / 5
Epoch 2 / 5
Epoch 3 / 5
Epoch 4 / 5
Epoch 5 / 5


In [ ]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

In [48]:
model = BertClassifier(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 18, vector_subclass=True)

train_epochs(epochs, tr, cv, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_GDRO')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1 / 5


KeyboardInterrupt: 

In [ ]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)